In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import difflib  #correct spelling automatic
from sklearn.feature_extraction.text import TfidfVectorizer   #to convert textual data into numerical values
from sklearn.metrics.pairwise import cosine_similarity

In [7]:
movies_data = pd.read_csv('/content/movies.csv')
movies_data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [8]:
movies_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 4803 non-null   int64  
 1   budget                4803 non-null   int64  
 2   genres                4775 non-null   object 
 3   homepage              1712 non-null   object 
 4   id                    4803 non-null   int64  
 5   keywords              4391 non-null   object 
 6   original_language     4803 non-null   object 
 7   original_title        4803 non-null   object 
 8   overview              4800 non-null   object 
 9   popularity            4803 non-null   float64
 10  production_companies  4803 non-null   object 
 11  production_countries  4803 non-null   object 
 12  release_date          4802 non-null   object 
 13  revenue               4803 non-null   int64  
 14  runtime               4801 non-null   float64
 15  spoken_languages     

In [22]:
movies_data.describe()

,index,budget,id,popularity,revenue,runtime,vote_average,vote_count
count,4803.000000,4.803000e+03,4803.000000,4803.000000,4.803000e+03,4801.000000,4803.000000,4803.000000
mean,2401.000000,2.904504e+07,57165.484281,21.492301,8.226064e+07,106.875859,6.092172,690.217989
std,1386.651002,4.072239e+07,88694.614033,31.816650,1.628571e+08,22.611935,1.194612,1234.585891
min,0.000000,0.000000e+00,5.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
25%,1200.500000,7.900000e+05,9014.500000,4.668070,0.000000e+00,94.000000,5.600000,54.000000
50%,2401.000000,1.500000e+07,14629.000000,12.921594,1.917000e+07,103.000000,6.200000,235.000000
75%,3601.500000,4.000000e+07,58610.500000,28.313505,9.291719e+07,118.000000,6.800000,737.000000
max,4802.000000,3.800000e+08,459488.000000,875.581305,2.787965e+09,338.000000,10.000000,13752.000000


In [23]:
movies_data.isnull().sum()

,0
index,0
budget,0
genres,0
homepage,3091
id,0
keywords,0
original_language,0
original_title,0
overview,0
popularity,0


In [24]:
selected_features=['genres','keywords','title','cast','director','popularity','overview']
print(selected_features)

['genres', 'keywords', 'title', 'cast', 'director', 'popularity', 'overview']


In [25]:
for i in selected_features:
  movies_data[i]=movies_data[i].fillna('')

In [26]:
combined_features=movies_data['genres']+' '+movies_data['keywords']+' '+movies_data['original_title']+' '+movies_data['cast']+' '+movies_data['director']+' '+str(movies_data['popularity'])+' '+movies_data['overview']

In [27]:
print(combined_features)

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  Newlyweds Edward Burns Kerry B...
4800    Comedy Drama Romance TV Movie date love at fir...
4801      Shanghai Calling Daniel Henney Eliza Coupe B...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object


In [28]:
vectorizer=TfidfVectorizer()

In [29]:
feature_vectors=vectorizer.fit_transform(combined_features)
print(feature_vectors)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 423221 stored elements and shape (4803, 31214)>
  Coords	Values
  (0, 563)	0.06204031765021313
  (0, 706)	0.07153390935539783
  (0, 9902)	0.08847635782601397
  (0, 24390)	0.0825259973577468
  (0, 10178)	0.08271666349535582
  (0, 6717)	0.15565248220673217
  (0, 5385)	0.16087509987660858
  (0, 10974)	0.12037390402945343
  (0, 25855)	0.2546488594106838
  (0, 29984)	0.09039589613130458
  (0, 5701)	0.17989704031497966
  (0, 25666)	0.1392399074442109
  (0, 2105)	0.21057249293353622
  (0, 24030)	0.11472958381769581
  (0, 30664)	0.1894673760827955
  (0, 31065)	0.15645333959108654
  (0, 23987)	0.1744930573719031
  (0, 25235)	0.16504814744386231
  (0, 30134)	0.15645333959108654
  (0, 26321)	0.12037390402945343
  (0, 15841)	0.17796832020984735
  (0, 18082)	0.12759136344412667
  (0, 23540)	0.15565248220673217
  (0, 14427)	0.08742348480574791
  (0, 4372)	0.13722139146604911
  :	:
  (4802, 9313)	0.07408389881208313
  (4802, 24519)	0.08236

In [30]:
similarity=cosine_similarity(feature_vectors)
print(similarity)

[[1.         0.04939305 0.04608761 ... 0.02907144 0.04150316 0.01702909]
 [0.04939305 1.         0.04442536 ... 0.05192336 0.04112373 0.02427535]
 [0.04608761 0.04442536 1.         ... 0.03373124 0.05727745 0.01948321]
 ...
 [0.02907144 0.05192336 0.03373124 ... 1.         0.04039589 0.04712718]
 [0.04150316 0.04112373 0.05727745 ... 0.04039589 1.         0.03620111]
 [0.01702909 0.02427535 0.01948321 ... 0.04712718 0.03620111 1.        ]]


In [31]:
print(similarity.shape)

(4803, 4803)


In [36]:
movie_name=input('Enter the movie name: ')

Enter the movie name: tangled


In [37]:
list_of_all_titles=movies_data['title'].tolist()

In [38]:
print(list_of_all_titles)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [39]:
close_match=difflib.get_close_matches(movie_name,list_of_all_titles)
print(close_match)

['Tangled', 'Triangle', 'Stranded']


In [40]:
close_match=close_match[0]
print(close_match)

Tangled


In [41]:
# finding the index of the movie with title

index_of_the_movie = movies_data[movies_data.title == close_match]['index'].values[0]
print(index_of_the_movie)

6


In [42]:
# getting a list of similar movies

similarity_score = list(enumerate(similarity[index_of_the_movie]))
print(similarity_score)

[(0, np.float64(0.02725481061854224)), (1, np.float64(0.03315823942067706)), (2, np.float64(0.021701757021449844)), (3, np.float64(0.04145448994536243)), (4, np.float64(0.04671898976750761)), (5, np.float64(0.0313171110731688)), (6, np.float64(0.9999999999999998)), (7, np.float64(0.040974574522163375)), (8, np.float64(0.04043514701726635)), (9, np.float64(0.05274630081175272)), (10, np.float64(0.03144381273466817)), (11, np.float64(0.02994823141235288)), (12, np.float64(0.03472976236468955)), (13, np.float64(0.0692185814440735)), (14, np.float64(0.052808978994953557)), (15, np.float64(0.04323779314522565)), (16, np.float64(0.03873459340675299)), (17, np.float64(0.048194310082564334)), (18, np.float64(0.0407509662359612)), (19, np.float64(0.04953062902253766)), (20, np.float64(0.05375885482072633)), (21, np.float64(0.03667391705107106)), (22, np.float64(0.024966278690602427)), (23, np.float64(0.04527391722294803)), (24, np.float64(0.03851256972327983)), (25, np.float64(0.040939288264552

In [43]:
len(similarity_score)

4803

In [44]:
# sorting the movies based on their similarity score

sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True)
print(sorted_similar_movies)

[(6, np.float64(0.9999999999999998)), (394, np.float64(0.11588439316765674)), (917, np.float64(0.11047602130039708)), (1511, np.float64(0.10733582085575039)), (3991, np.float64(0.10732206533294333)), (1984, np.float64(0.1068532697298445)), (4659, np.float64(0.10570831172438676)), (538, np.float64(0.10249506920559627)), (498, np.float64(0.1020431124891096)), (950, np.float64(0.09973793282517886)), (391, np.float64(0.09936531648613958)), (39, np.float64(0.09859063504444673)), (3131, np.float64(0.09729928059448895)), (4326, np.float64(0.09580241704385764)), (1249, np.float64(0.09562981273636673)), (585, np.float64(0.09495283771514962)), (403, np.float64(0.09388717876305834)), (2309, np.float64(0.09354878587042624)), (734, np.float64(0.09210315544084043)), (4795, np.float64(0.09159221935323239)), (4123, np.float64(0.0903461780051289)), (1668, np.float64(0.09024255283421939)), (1669, np.float64(0.08969778156459585)), (330, np.float64(0.08896649636686604)), (420, np.float64(0.088906132427709

In [45]:
# print the name of similar movies based on the index

print('Movies suggested for you : \n')

i = 1

for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = movies_data[movies_data.index==index]['title'].values[0]
  if (i<30):
    print(i, '.',title_from_index)
    i+=1

Movies suggested for you : 

1 . Tangled
2 . Tower Heist
3 . Into the Woods
4 . Dreamer: Inspired By a True Story
5 . The Black Stallion
6 . The Thief and the Cobbler
7 . Horse Camp
8 . Hostage
9 . Hidalgo
10 . The Negotiator
11 . Enchanted
12 . TRON: Legacy
13 . Of Horses and Men
14 . The Raid
15 . Torque
16 . War Horse
17 . Last Action Hero
18 . Out of Inferno
19 . The Tale of Despereaux
20 . Bang
21 . Roadside Romeo
22 . Miss Potter
23 . The Promise
24 . The Lord of the Rings: The Two Towers
25 . Hellboy II: The Golden Army
26 . Shrek 2
27 . Hansel and Gretel Get Baked
28 . Oliver!
29 . Silverado


In [46]:
movie_name = input(' Enter your favourite movie name : ')

list_of_all_titles = movies_data['title'].tolist()

find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)

close_match = find_close_match[0]

index_of_the_movie = movies_data[movies_data.title == close_match]['index'].values[0]

similarity_score = list(enumerate(similarity[index_of_the_movie]))

sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True)

print('Movies suggested for you : \n')

i = 1

for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = movies_data[movies_data.index==index]['title'].values[0]
  if (i<30):
    print(i, '.',title_from_index)
    i+=1

 Enter your favourite movie name : Tanggked
Movies suggested for you : 

1 . Tangled
2 . Tower Heist
3 . Into the Woods
4 . Dreamer: Inspired By a True Story
5 . The Black Stallion
6 . The Thief and the Cobbler
7 . Horse Camp
8 . Hostage
9 . Hidalgo
10 . The Negotiator
11 . Enchanted
12 . TRON: Legacy
13 . Of Horses and Men
14 . The Raid
15 . Torque
16 . War Horse
17 . Last Action Hero
18 . Out of Inferno
19 . The Tale of Despereaux
20 . Bang
21 . Roadside Romeo
22 . Miss Potter
23 . The Promise
24 . The Lord of the Rings: The Two Towers
25 . Hellboy II: The Golden Army
26 . Shrek 2
27 . Hansel and Gretel Get Baked
28 . Oliver!
29 . Silverado
